In [3]:
import numpy as np
import pandas as pd
from os import listdir
import networkx as nx
from copy import deepcopy
import matplotlib.pyplot as plt
import Compute_Scaffold_Coverages_Utility as cc
from importlib import reload


In [4]:
graphspath = '/Users/harihara/Mount-2/MetaCarvel_paper/hmp_scaffolds/stool/'
covpath = '/Users/harihara/Research-Activities/Data/Binnacle-Op/genomecov_d/'

In [6]:
sample = 'SRS054590'

In [8]:
Graph_path= graphspath+sample+'/'+sample+'_scaffolds/oriented.gml'
df_coverage, G =cc.Load_Read_Coverage_and_Assembly_Graph(Graph_path, covpath+sample+'.txt')

<class 'pandas.core.frame.DataFrame'>
Index: 14841277 entries, 10 to contig-100_9971.9971
Data columns (total 2 columns):
Loc         int64
coverage    int32
dtypes: int32(1), int64(1)
memory usage: 283.1+ MB
None


In [25]:
reload(cc)
(df_coords_before_delinking, 
 df_coords_after_delinking,
 df_coverage_scaffolds_before_delinking, 
 df_coverage_scaffolds_after_delinking) = cc.Write_Coverage_Outputs(G, df_coverage)


1011
Debug----> ['contig-100_1464.26502', '10', '1706', 'contig-100_1542.26580']
Debug----> contig-100_1542.26580
Debug----> 1
contig-100_1542.26580 Neighbors ['1706']
('contig-100_1542.26580', '1706') EB -359 1765
1706 1892 3657
1706 Neighbors ['10', 'contig-100_1464.26502', 'contig-100_1542.26580']
('1706', '10') EB -205 7589
10 3452 11041
('1706', 'contig-100_1464.26502') EB -1514 2375
contig-100_1464.26502 2143 4518
('contig-100_1542.26580', '1706') EB -359 2251
contig-100_1542.26580 0 2251
contig-100_1464.26502 Neighbors ['1706']
('1706', 'contig-100_1464.26502') EB -1514 1765
1706 1892 3657
10 Neighbors ['1706']
('1706', '10') EB -205 1765
1706 1892 3657
{'contig-100_1542.26580': (0, 2251), '1706': (1892, 3657), '10': (3452, 11041), 'contig-100_1464.26502': (2143, 4518)}
dict_keys(['contig-100_1542.26580', '1706', '10', 'contig-100_1464.26502'])
1 4 1 

Debug----> ['1330', 'contig-100_613.28495', '1788', '770', '679', '1429', 'contig-100_676.29685', '994', '138', 'contig-100_748.

1788 Neighbors []
{'1788': (0, 6797)}
dict_keys(['1788'])
1429 Neighbors []
{'1429': (0, 3274)}
dict_keys(['1429'])
contig-100_748.29757 Neighbors []
{'contig-100_748.29757': (0, 776)}
dict_keys(['contig-100_748.29757'])
719 Neighbors ['contig-100_539.29548']
('719', 'contig-100_539.29548') EB -296 1727
contig-100_539.29548 55936 57663
contig-100_539.29548 Neighbors ['719']
('719', 'contig-100_539.29548') EB -296 56232
719 0 56232
{'719': (0, 56232), 'contig-100_539.29548': (55936, 57663)}
dict_keys(['719', 'contig-100_539.29548'])
Debug----> ['contig-100_1866.1866', '1001']
Debug----> 1001
Debug----> 3
1001 Neighbors ['contig-100_1866.1866']
('1001', 'contig-100_1866.1866') EB -121 1651
contig-100_1866.1866 4497 6148
contig-100_1866.1866 Neighbors ['1001']
('1001', 'contig-100_1866.1866') EB -121 4618
1001 0 4618
{'1001': (0, 4618), 'contig-100_1866.1866': (4497, 6148)}
dict_keys(['1001', 'contig-100_1866.1866'])
3 2 1 

Debug----> ['contig-100_9755.9755', 'contig-100_1045.26083', '98

406 Neighbors ['contig-100_805.28687']
('406', 'contig-100_805.28687') BB -136 1714
contig-100_805.28687 -136 1578
contig-100_805.28687 Neighbors ['contig-100_1399.26437', '406']
('contig-100_805.28687', 'contig-100_1399.26437') EB -286 2513
contig-100_1399.26437 1292 3805
('406', 'contig-100_805.28687') BB -136 35288
406 0 -35288
contig-100_1399.26437 Neighbors ['contig-100_805.28687', '1036']
('contig-100_805.28687', 'contig-100_1399.26437') EB -286 1714
contig-100_805.28687 -136 1578
('1036', 'contig-100_1399.26437') BB -2436 28559
1036 3728 -24831
1036 Neighbors ['456', 'contig-100_1399.26437']
('456', '1036') EE -259 41064
456 -65636 -24572
('1036', 'contig-100_1399.26437') BB -2436 2513
contig-100_1399.26437 1292 3805
456 Neighbors ['1036']
('456', '1036') EE -259 28559
1036 3728 -24831
{'406': (65636, 30348), 'contig-100_805.28687': (65500, 67214), 'contig-100_1399.26437': (66928, 69441), '1036': (69364, 40805), '456': (0, 41064)}
dict_keys(['406', 'contig-100_805.28687', 'conti

('1692', 'contig-100_442.25480') EB -334 9214
contig-100_442.25480 325049 334263
('532', '1692') BB -48115 107921
532 323704 215783
('contig-100_756.29765', '1692') BB -381 758
contig-100_756.29765 275970 275212
contig-100_756.29765 Neighbors ['contig-100_894.29903', 'contig-100_910.29919', '1692']
('contig-100_894.29903', 'contig-100_756.29765') EE -170 555
contig-100_894.29903 274827 275382
('contig-100_910.29919', 'contig-100_756.29765') BE -201 538
contig-100_910.29919 275413 274875
('contig-100_756.29765', '1692') BB -381 49794
1692 275589 325383
contig-100_910.29919 Neighbors ['contig-100_440.29449', 'contig-100_756.29765']
('contig-100_440.29449', 'contig-100_910.29919') EE -169 2712
contig-100_440.29449 272332 275044
('contig-100_910.29919', 'contig-100_756.29765') BE -201 758
contig-100_756.29765 275970 275212
contig-100_440.29449 Neighbors ['contig-100_561.29570', 'contig-100_633.29642', 'contig-100_910.29919']
('contig-100_561.29570', 'contig-100_440.29449') BB -165 1596
con

19 135 4 

contig-100_328.29337 Neighbors ['1344', 'contig-100_178.29187']
('contig-100_328.29337', '1344') BB -269 1036
1344 -269 767
('contig-100_328.29337', 'contig-100_178.29187') BB -153 14634
contig-100_178.29187 -153 14481
contig-100_178.29187 Neighbors ['contig-100_328.29337', 'contig-100_357.29366']
('contig-100_328.29337', 'contig-100_178.29187') BB -153 5743
contig-100_328.29337 0 -5743
('contig-100_178.29187', 'contig-100_357.29366') EE -138 4730
contig-100_357.29366 19073 14343
contig-100_357.29366 Neighbors ['1785', 'contig-100_178.29187']
('contig-100_357.29366', '1785') BE -153 16223
1785 35143 18920
('contig-100_178.29187', 'contig-100_357.29366') EE -138 14634
contig-100_178.29187 -153 14481
1785 Neighbors ['contig-100_357.29366', 'contig-100_318.29327']
('contig-100_357.29366', '1785') BE -153 4730
contig-100_357.29366 19073 14343
('1785', 'contig-100_318.29327') BE -634 6083
contig-100_318.29327 40592 34509
contig-100_318.29327 Neighbors ['contig-100_353.29362', '17

...

contig-100_4109.4109 Neighbors []
{'contig-100_4109.4109': (0, 1104)}
dict_keys(['contig-100_4109.4109'])
Debug----> ['contig-100_411.29420']
Debug----> contig-100_411.29420
Debug----> 773
contig-100_411.29420 Neighbors []
{'contig-100_411.29420': (0, 3189)}
dict_keys(['contig-100_411.29420'])
Debug----> ['contig-100_4119.4119']
Debug----> contig-100_4119.4119
Debug----> 774
contig-100_4119.4119 Neighbors []
{'contig-100_4119.4119': (0, 1102)}
dict_keys(['contig-100_4119.4119'])
Debug----> ['contig-100_4123.4123', 'contig-100_5402.5402']
Debug----> contig-100_4123.4123
Debug----> 775
contig-100_4123.4123 Neighbors ['contig-100_5402.5402']
('contig-100_4123.4123', 'contig-100_5402.5402') EE -124 948
contig-100_5402.5402 1926 978
contig-100_5402.5402 Neighbors ['contig-100_4123.4123']
('contig-100_4123.4123', 'contig-100_5402.5402') EE -124 1102
contig-100_4123.4123 0 1102
{'contig-100_4123.4123': (0, 1102), 'contig-100_5402.5402': (1926, 978)}
dict_keys(['contig-100_4123.4123', 'contig-

Debug----> ['contig-100_927.28809', 'contig-100_5100.5100']
Debug----> contig-100_5100.5100
Debug----> 825
contig-100_5100.5100 Neighbors ['contig-100_927.28809']
('contig-100_5100.5100', 'contig-100_927.28809') EE -94 756
contig-100_927.28809 1641 885
contig-100_927.28809 Neighbors ['contig-100_5100.5100']
('contig-100_5100.5100', 'contig-100_927.28809') EE -94 979
contig-100_5100.5100 0 979
{'contig-100_5100.5100': (0, 979), 'contig-100_927.28809': (1641, 885)}
dict_keys(['contig-100_5100.5100', 'contig-100_927.28809'])
825 2 2 

contig-100_927.28809 Neighbors []
{'contig-100_927.28809': (756, 0)}
dict_keys(['contig-100_927.28809'])
contig-100_5100.5100 Neighbors []
{'contig-100_5100.5100': (0, 979)}
dict_keys(['contig-100_5100.5100'])
Debug----> ['contig-100_511.29520']
Debug----> contig-100_511.29520
Debug----> 826
contig-100_511.29520 Neighbors []
{'contig-100_511.29520': (0, 1893)}
dict_keys(['contig-100_511.29520'])
Debug----> ['contig-100_6315.6315', 'contig-100_519.29528']
Deb

Debug----> ['contig-100_6047.6047']
Debug----> contig-100_6047.6047
Debug----> 871
contig-100_6047.6047 Neighbors []
{'contig-100_6047.6047': (0, 885)}
dict_keys(['contig-100_6047.6047'])
Debug----> ['contig-100_7020.7020', 'contig-100_6050.6050']
Debug----> contig-100_6050.6050
Debug----> 872
contig-100_6050.6050 Neighbors ['contig-100_7020.7020']
('contig-100_6050.6050', 'contig-100_7020.7020') EB -163 807
contig-100_7020.7020 722 1529
contig-100_7020.7020 Neighbors ['contig-100_6050.6050']
('contig-100_6050.6050', 'contig-100_7020.7020') EB -163 885
contig-100_6050.6050 0 885
{'contig-100_6050.6050': (0, 885), 'contig-100_7020.7020': (722, 1529)}
dict_keys(['contig-100_6050.6050', 'contig-100_7020.7020'])
872 2 1 

Debug----> ['contig-100_609.609']
Debug----> contig-100_609.609
Debug----> 873
contig-100_609.609 Neighbors []
{'contig-100_609.609': (0, 2556)}
dict_keys(['contig-100_609.609'])
Debug----> ['contig-100_6099.6099']
Debug----> contig-100_6099.6099
Debug----> 874
contig-100

914 2 1 

Debug----> ['contig-100_700.25738']
Debug----> contig-100_700.25738
Debug----> 915
contig-100_700.25738 Neighbors []
{'contig-100_700.25738': (0, 6447)}
dict_keys(['contig-100_700.25738'])
Debug----> ['contig-100_704.704']
Debug----> contig-100_704.704
Debug----> 916
contig-100_704.704 Neighbors []
{'contig-100_704.704': (0, 2434)}
dict_keys(['contig-100_704.704'])
Debug----> ['contig-100_7059.7059']
Debug----> contig-100_7059.7059
Debug----> 917
contig-100_7059.7059 Neighbors []
{'contig-100_7059.7059': (0, 804)}
dict_keys(['contig-100_7059.7059'])
Debug----> ['contig-100_708.708']
Debug----> contig-100_708.708
Debug----> 918
contig-100_708.708 Neighbors []
{'contig-100_708.708': (0, 2431)}
dict_keys(['contig-100_708.708'])
Debug----> ['contig-100_7081.7081']
Debug----> contig-100_7081.7081
Debug----> 919
contig-100_7081.7081 Neighbors []
{'contig-100_7081.7081': (0, 802)}
dict_keys(['contig-100_7081.7081'])
Debug----> ['contig-100_7097.7097']
Debug----> contig-100_7097.7097

contig-100_935.935 Neighbors []
{'contig-100_935.935': (0, 2189)}
dict_keys(['contig-100_935.935'])
Debug----> ['contig-100_939.29948']
Debug----> contig-100_939.29948
Debug----> 986
contig-100_939.29948 Neighbors []
{'contig-100_939.29948': (0, 507)}
dict_keys(['contig-100_939.29948'])
Debug----> ['contig-100_94.94']
Debug----> contig-100_94.94
Debug----> 987
contig-100_94.94 Neighbors []
{'contig-100_94.94': (0, 4597)}
dict_keys(['contig-100_94.94'])
Debug----> ['contig-100_9472.9472']
Debug----> contig-100_9472.9472
Debug----> 988
contig-100_9472.9472 Neighbors []
{'contig-100_9472.9472': (0, 661)}
dict_keys(['contig-100_9472.9472'])
Debug----> ['contig-100_948.25986']
Debug----> contig-100_948.25986
Debug----> 989
contig-100_948.25986 Neighbors []
{'contig-100_948.25986': (0, 4490)}
dict_keys(['contig-100_948.25986'])
Debug----> ['contig-100_949.949']
Debug----> contig-100_949.949
Debug----> 990
contig-100_949.949 Neighbors []
{'contig-100_949.949': (0, 2176)}
dict_keys(['contig-10

In [12]:
conn_comps = list(nx.weakly_connected_component_subgraphs(G))

In [13]:
for i in range(0, len(conn_comps)):
    if '1795' in conn_comps[i].nodes():
        print(i)

206


In [16]:
temp = deepcopy(conn_comps[206])


In [18]:
temp.remove_edge('1795','contig-100_546.29555')

In [20]:
list(nx.weakly_connected_component_subgraphs(temp))

In [22]:
e = set([(1,2),(2,3)])